In [1]:
%load_ext jupyter_black

In [2]:
import os
import uuid
from pathlib import Path
from warnings import warn
from datetime import datetime
from typing import Callable, Union, TypedDict

import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
from geopandas import GeoDataFrame
from requests import Session, HTTPError


PROBSEVERE_URL_TEMPLATE = (
    "https://mtarchive.geol.iastate.edu/%Y/%m/%d/mrms/ncep/ProbSevere/MRMS_PROBSEVERE_%Y%m%d_%H%M00.json"
)

FILE_OUT_DIR = os.path.abspath("../../bucket")

TimeLike = Union[datetime, str, pd.Timestamp]


class Feature(TypedDict):
    properties: dict[str, Union[float, int, str]]


class FeatureCollection(TypedDict):
    validTime: str
    features: list[Feature]

In [3]:
cluster = LocalCluster()  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default
client

2022-08-05 07:30:03,826 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-rzlwic6j', purging
2022-08-05 07:30:03,826 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-_lwwzptv', purging
2022-08-05 07:30:03,826 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-4pt3shpj', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 3
Total threads: 6,Total memory: 15.58 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41841,Workers: 3
Dashboard: http://127.0.0.1:8787/status,Total threads: 6
Started: Just now,Total memory: 15.58 GiB
Comm: tcp://127.0.0.1:45613,Total threads: 2
Dashboard: http://127.0.0.1:42163/status,Memory: 5.19 GiB
Nanny: tcp://127.0.0.1:34493,


In [4]:
__uuid = uuid.uuid4()


def __batch_id(validtime: datetime) -> Callable[[int], str]:
    def wrapper(n: int) -> str:
        return f"probsevere-{n}-{validtime.isoformat(timespec='minutes')}-{__uuid}.parquet"

    return wrapper


def __features(features: list[Feature]) -> pd.DataFrame:
    return GeoDataFrame.from_features(features)


def __bounds(
    df: pd.DataFrame,
    drop_columns: list[str] = ["MAXRC_EMISS", "MAXRC_ICECF", "AVG_BEAM_HGT", "geometry"],
) -> pd.DataFrame:
    return pd.concat((df, df["geometry"].bounds), axis=1).drop(columns=drop_columns)

In [5]:
def direct_to_parquet(
    path_dir: Path,
    start: TimeLike,
    end: TimeLike,
    freq: str = "2min",
) -> None:
    """scrape data from the iastate archive over a daterange"""
    # create a DatetimeIndex using the the function arguments and format the urls using the url template
    urls = pd.date_range(start=start, end=end, freq=freq)
    # using Session as a context manager
    with Session() as session:
        # iterating over all of the urls
        for url in urls.strftime(PROBSEVERE_URL_TEMPLATE):
            # using a try/catch block in the event the download fails
            try:
                # with our session make a get request, r is a response object
                r = session.get(url, stream=True)
                # in the event of a non 200 status code we'll raise a HTTPError and trigger the except block
                r.raise_for_status()
            # if there was an error downloading, continue
            except (ConnectionError, HTTPError):
                warn(f"error downloading {url}")
                continue
            fc: FeatureCollection = r.json()

            features = fc["features"]
            # in the event no storms were record, continue
            if not features:
                continue
            #
            df = __features(features).pipe(__bounds).astype(np.float32)

            valid_time = datetime.strptime(fc["validTime"], "%Y%m%d_%H%M%S %Z")
            # set the valid time
            df["valid_time"] = valid_time

            dd.from_pandas(df, chunksize=256).to_parquet(
                path_dir,
                engine="pyarrow",
                append=True,
                name_function=__batch_id(valid_time),
                ignore_divisions=True,
            )

In [6]:
if __name__ == "__main__":
    # NOTE: the first time this ran it collected files from 2022-03-01T00:00:00Z -> 2022-05-14T15:54:00Z
    direct_to_parquet(
        Path(FILE_OUT_DIR),
        # start="2022-03-01T00:00:00Z",
        start="2022-05-14T15:54",
        end="2022-08-4T00:00",
    )

/tmp/ipykernel_8980/4055683049.py:22: UserWarning: error downloading https://mtarchive.geol.iastate.edu/2022/05/15/mrms/ncep/ProbSevere/MRMS_PROBSEVERE_20220515_061600.json
  warn(f"error downloading {url}")
/tmp/ipykernel_8980/4055683049.py:22: UserWarning: error downloading https://mtarchive.geol.iastate.edu/2022/05/15/mrms/ncep/ProbSevere/MRMS_PROBSEVERE_20220515_062000.json
  warn(f"error downloading {url}")
/tmp/ipykernel_8980/4055683049.py:22: UserWarning: error downloading https://mtarchive.geol.iastate.edu/2022/05/15/mrms/ncep/ProbSevere/MRMS_PROBSEVERE_20220515_062200.json
  warn(f"error downloading {url}")
/tmp/ipykernel_8980/4055683049.py:22: UserWarning: error downloading https://mtarchive.geol.iastate.edu/2022/05/15/mrms/ncep/ProbSevere/MRMS_PROBSEVERE_20220515_062400.json
  warn(f"error downloading {url}")
/tmp/ipykernel_8980/4055683049.py:22: UserWarning: error downloading https://mtarchive.geol.iastate.edu/2022/05/15/mrms/ncep/ProbSevere/MRMS_PROBSEVERE_20220515_062600.

KeyboardInterrupt: 